In [5]:
# Set project directory
import os
import sys


def project_dir():
    notebook_path = %pwd
    repo_name = "worldbank_data_exploration"
    repo_folder = notebook_path.split(repo_name)[0]
    return os.path.join(repo_folder, repo_name)


pwd = os.getenv("PWD", project_dir())
os.environ["PWD"] = pwd
sys.path.append(pwd)

In [6]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

from data_sources.get import get_indicators

In [59]:
df = get_indicators(
    [
        "SP.POP.GROW",
        "FP.CPI.TOTL.ZG",
        "SP.DYN.LE00.IN",
        "NE.EXP.GNFS.ZS",
        "NY.GDP.MKTP.KD.ZG",
        "SL.UEM.TOTL.ZS",
        "NV.AGR.TOTL.ZS",
        "EG.ELC.ACCS.ZS",
        "AG.LND.FRST.ZS",
        "SH.DYN.MORT",
        "NY.GDP.TOTL.RT.ZS",
        "SP.DYN.TFRT.IN",
        "EN.URB.LCTY.UR.ZS",
        "TG.VAL.TOTL.GD.ZS",
        "MS.MIL.XPND.GD.ZS",
    ]
)

print(df["Indicator Name"].unique())

['Population growth (annual %)' 'Inflation, consumer prices (annual %)'
 'Life expectancy at birth, total (years)'
 'Exports of goods and services (% of GDP)' 'GDP growth (annual %)'
 'Unemployment, total (% of total labor force) (modeled ILO estimate)'
 'Agriculture, forestry, and fishing, value added (% of GDP)'
 'Access to electricity (% of population)' 'Forest area (% of land area)'
 'Mortality rate, under-5 (per 1,000 live births)'
 'Total natural resources rents (% of GDP)'
 'Fertility rate, total (births per woman)'
 'Population in the largest city (% of urban population)'
 'Merchandise trade (% of GDP)' 'Military expenditure (% of GDP)']


In [60]:
df = df.pivot_table(
    values="Value", index="Year", columns=["Indicator Name", "Country Name"]
)

In [61]:
df_nans = df.loc[slice(2000, 2019)].isnull().sum().reset_index()

In [62]:
countries_with_nans = df_nans[df_nans[0] > 2]["Country Name"].unique()
countries_with_nans

array(['Afghanistan', 'Algeria', 'Congo, Rep.', 'Equatorial Guinea',
       'Guinea-Bissau', 'Iraq', 'Kiribati', "Korea, Dem. People's Rep.",
       'Lebanon', 'Liberia', 'Malaysia', 'Sierra Leone', 'South Sudan',
       'Turkey', 'Armenia', 'Aruba', 'Bahrain', 'Barbados', 'Bermuda',
       'Cayman Islands', 'Central African Republic', 'Djibouti',
       'Eritrea', 'French Polynesia', 'Greenland', 'Isle of Man',
       'Kosovo', 'Libya', 'Liechtenstein', 'Maldives', 'Nauru',
       'San Marino', 'Sint Maarten (Dutch part)', 'Solomon Islands',
       'Somalia', 'St. Lucia', 'Tuvalu', 'Venezuela, RB', 'Curacao',
       'Dominica', 'Ethiopia', 'Guyana', 'Lao PDR', 'Lesotho', 'Myanmar',
       'Papua New Guinea', 'Suriname', 'Yemen, Rep.', 'Andorra',
       'Marshall Islands', 'Palau', 'St. Kitts and Nevis', 'Sudan',
       'World', 'American Samoa', 'Channel Islands', 'Guam',
       'New Caledonia', 'Northern Mariana Islands',
       'Turks and Caicos Islands', 'Virgin Islands (U.S.)',
  

In [63]:
df_cleared = df.loc[2000:2019].stack()
df_cleared.drop(index=countries_with_nans, level=1, inplace=True)

In [64]:
df_cleared.bfill(inplace=True)
df_cleared.ffill(inplace=True)

In [65]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_cleared.values)
df_scaled = pd.DataFrame(
    data=data_scaled, index=df_cleared.index, columns=df_cleared.columns
)
df_scaled.head()

Indicator Name                    Access to electricity (% of population)  \
Year Country Name                                                           
2000 Africa Eastern and Southern                                -2.510390   
     Africa Western and Central                                 -1.938826   
     Albania                                                     0.624115   
     Angola                                                      0.524322   
     Antigua and Barbuda                                         0.524322   

Indicator Name                    Agriculture, forestry, and fishing, value added (% of GDP)  \
Year Country Name                                                                              
2000 Africa Eastern and Southern                                           0.337255            
     Africa Western and Central                                            1.317580            
     Albania                                                               1.691805            
     Angola                                                               -0.447382            
     Antigua and Barbuda                                                  -0.908996            

Indicator Name                    Exports of goods and services (% of GDP)  \
Year Country Name                                                            
2000 Africa Eastern and Southern                                 -0.516288   
     Africa Western and Central                                  -0.142854   
     Albania                                                     -0.730713   
     Angola                                                       1.807223   
     Antigua and Barbuda                                          2.132473   

Indicator Name                    Fertility rate, total (births per woman)  \
Year Country Name                                                            
2000 Africa Eastern and Southern                                  2.184692   
     Africa Western and Central                                   2.526298   
     Albania                                                     -0.365591   
     Angola                                                       2.980123   
     Antigua and Barbuda                                         -0.333492   

Indicator Name                    Forest area (% of land area)  \
Year Country Name                                                
2000 Africa Eastern and Southern                      0.133863   
     Africa Western and Central                      -0.512883   
     Albania                                         -0.174435   
     Angola                                           1.614304   
     Antigua and Barbuda                             -0.519045   

Indicator Name                    GDP growth (annual %)  \
Year Country Name                                         
2000 Africa Eastern and Southern              -0.157276   
     Africa Western and Central               -0.007551   
     Albania                                   0.887072   
     Angola                                   -0.196976   
     Antigua and Barbuda                       0.680161   

Indicator Name                    Inflation, consumer prices (annual %)  \
Year Country Name                                                         
2000 Africa Eastern and Southern                               0.429700   
     Africa Western and Central                               -0.257431   
     Albania                                                  -0.538223   
     Angola                                                   36.241832   
     Antigua and Barbuda                                      -0.456528   

Indicator Name                    Life expectancy at birth, total (years)  \
Year Country Name                                                           
2000 Africa Eastern and Southern                                -2.472460   
     Africa Western and Central                            